---
---
---
---

# Start Here!

In [1]:
import os
import wave
import pyaudio
import time
import math
from glob import glob

import socket
from struct import pack, unpack

UDP_IP = '192.168.1.34'
UDP_PORT = 161

In [2]:
def turntable(COMMAND=4, ANGLE=0):
    if COMMAND==3 or COMMAND==4:
        ANGLE=0

    MESSAGE = pack('hh', COMMAND, int(ANGLE*10))

    """
    print("UDP target IP: {}".format(UDP_IP))
    print("UDP target port: {}".format(UDP_PORT))
    print('SENT : {}'.format(MESSAGE))
    """

    sock = socket.socket(socket.AF_INET, # Internet
                        socket.SOCK_DGRAM) #UDP
    sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))
    DATA, ADDR = sock.recvfrom(1024)

    if len(DATA) is not None:
        #print('RECEIVED : {}'.format(DATA))

        if COMMAND==1 or COMMAND==2 or COMMAND==3:
            if DATA[0]==0x33:
                print('UDP COMMAND : SUCCESSFUL')
            elif DATA[0]==0x66:
                print('UDP COMMAND : UNSUCCESSFUL')
            else:
                print('ERROR_1')

        elif COMMAND==4:
            if DATA[0]==0x05:
                READING = unpack('>hh', DATA[1:5])[1]/10
                print('TURNTABLE STOPPED')
                print('CURRENT ANGLE : {}\n'.format(str(READING)))
            elif DATA[0]==0x04:
                print('TURNTABLE STILL IN MOTION')
            else:
                print('ERROR_2')

        else:
            print('ERROR_3')
turntable()

TURNTABLE STOPPED
CURRENT ANGLE : 0.0



In [3]:
class_list = ['cough', 'crying', 'falling', 'shout', 'speech']
#class_list = ['falling']

source_parent_dir = '..\\Rishabh\\Motu_Ready_Files\\'


def recorder(OUT_DIR, SP):
    
    print('Channel Number : {}'.format(SP))

    
    for cls in class_list:
        print(cls)
        source_child_dir = source_parent_dir + cls + '\\'
        wav_list = glob(source_child_dir + '*\\*' + str(SP) + '.wav')
        
        for wav_dir in wav_list:
            data_play = wave.open(wav_dir, 'rb')

            print('Wav File : {}'.format(wav_dir))

            p = pyaudio.PyAudio()

            In_Channel = data_play.getnchannels()
            In_Sample = data_play.getsampwidth()
            In_Frames = data_play.getnframes()
            In_Rate = data_play.getframerate()
            In_Duration = In_Frames / float(In_Rate)

            def callback(in_data, frame_count, time_info, flag):
                data = data_play.readframes(frame_count)
                return (data, pyaudio.paContinue)

            #Start Recording
            CHUNK = 1024
            FORMAT = pyaudio.paInt16
            CHANNELS = 8
            RATE = 44100
            RECORD_SECONDS = In_Duration

            stream_in = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

            frames=[]
            front_buffer = int(RATE/CHUNK*0.5)

            for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                data=stream_in.read(CHUNK)
                frames.append(data)
                #print('.', end='')
                if i == front_buffer:        
                    stream_out = p.open(format=p.get_format_from_width(data_play.getsampwidth()),
                            channels=10,
                            rate=RATE,
                            output=True,
                            stream_callback=callback,
                            output_device_index=7)

            # start the stream (4)
            stream_out.start_stream()

            # wait for stream to finish (5)
            while stream_out.is_active():
                time.sleep(0.1)

            # stop stream (6)
            stream_out.stop_stream()
            stream_out.close()
            data_play.close()

            stream_in.stop_stream()
            stream_in.close()

            # close PyAudio (7)
            p.terminate()
            
            OUT_DIR_T = OUT_DIR + 'ch_' + str(SP) + '\\' + cls + '\\'
            if not os.path.exists(OUT_DIR_T):
                os.makedirs(OUT_DIR_T)

            data_rec = wave.open(OUT_DIR_T + wav_dir.split('\\')[-2] +'.wav', 'wb')
            data_rec.setnchannels(CHANNELS)
            data_rec.setsampwidth(p.get_sample_size(FORMAT))
            data_rec.setframerate(RATE)
            data_rec.writeframes(b''.join(frames))
            data_rec.close()


In [4]:
import time, datetime

# Change Elev

In [5]:
start = time.time()

ANGLE=0
ELEVATION = 1520 # 0, 1520
    
OUT_DIR= '..\\Rishabh\\Recorded_Tracks\\ELEV_' + str(ELEVATION) + '_PADDED\\' + 'ANG_' + str(ANGLE) + '\\'

print(OUT_DIR)

for i in range(6):
    recorder(OUT_DIR, i+1)

end = time.time()

print("Time Elapsed : {} ".format(datetime.timedelta(0, end-start)))

..\Rishabh\Recorded_Tracks\ELEV_1520_PADDED\ANG_0\
Channel Number : 1
cough
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0016_0001\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0035_0001\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0041_0002\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0072_0003\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0131\ch_1.wav
crying
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0041_0000\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0074_0006\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0084_0006\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0085_0000\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0089_0000\ch_1.wav
falling
Wav File : ..\Rishabh\Motu_Ready_Files\falling\freesound_025_0004\ch_1.wav
Wav File : ..\Rishabh\Motu_Ready_Files\falling\soundsnap_0036_0000\ch_1.wav
Wav File : ..\Rish

In [5]:
##TEST
ANGLE=0
ELEVATION = 0 #0, 1000, 1520
for ANGLE in range(1):
    for REP in range(1,4):
        #OUT_DIR= 'ELEV_' + str(ELEVATION) + '_PADDED/' + 'ANG_' + str(ANGLE) + '/' + 'REP_' + str(REP) + '/'
        OUT_DIR= 'NEW_' + str(ELEVATION) + '/' + 'ANG_' + str(ANGLE) + '/' + 'REP_' + str(REP) + '/'

        print(OUT_DIR)

        if not os.path.exists(OUT_DIR):
            os.makedirs(OUT_DIR)

        recorder(OUT_DIR)
        time.sleep(1)
    
    #turntable(1,5)
    time.sleep(3)
    #turntable()

NEW_0/ANG_0/REP_1/
CHANNEL : 1/6
NEW_0/ANG_0/REP_2/
CHANNEL : 1/6
NEW_0/ANG_0/REP_3/
CHANNEL : 1/6


In [6]:
ANGLE_LIST=[5, 12, 19, 19, 20, 20, 37, 41, 43, 48, 51, 55]
print(len(ANGLE_LIST), ANGLE_LIST)
SPEAKER = [3, 0, 2, 4, 1, 5, 5, 2, 4, 3, 1, 0]
print(len(SPEAKER), SPEAKER)

12 [5, 12, 19, 19, 20, 20, 37, 41, 43, 48, 51, 55]
12 [3, 0, 2, 4, 1, 5, 5, 2, 4, 3, 1, 0]


# Change Elev

In [7]:
start=time.time()
ANGLE=0
ELEVATION = 1520 # 0, 1520

for i in range(0, len(ANGLE_LIST)):
    a_str =time.time()
    
    OUT_DIR= '..\\Rishabh\\Recorded_Tracks\\ELEV_' + str(ELEVATION) + '_PADDED\\' + 'ANG_' + str(ANGLE_LIST[i]) + '\\'

    print(OUT_DIR)
    
    if i == 0:
        DELTA = ANGLE_LIST[0]
    else:
        DELTA = ANGLE_LIST[i]- ANGLE_LIST[i-1]
    
    print(DELTA)
    if not DELTA == 0:
        DELAY = math.ceil(3/5*DELTA)
        turntable(1,DELTA)
        time.sleep(DELAY)
    
    turntable()    
    recorder(OUT_DIR, SPEAKER[i]+1)
    
    a_end = time.time()
    print("Time Elapsed for ANG_{}: {} ".format(ANGLE_LIST[i],datetime.timedelta(0, a_end-a_str)))
    
end = time.time()
print("Total Time Elapsed : {} ".format(datetime.timedelta(0, end-start)))

..\Rishabh\Recorded_Tracks\ELEV_1520_PADDED\ANG_5\
5
UDP COMMAND : SUCCESSFUL
TURNTABLE STOPPED
CURRENT ANGLE : 5.0

Channel Number : 4
cough
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0016_0001\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0035_0001\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0041_0002\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0072_0003\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\cough\soundsnap_0131\ch_4.wav
crying
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0041_0000\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0074_0006\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0084_0006\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0085_0000\ch_4.wav
Wav File : ..\Rishabh\Motu_Ready_Files\crying\soundsnap_0089_0000\ch_4.wav
falling
Wav File : ..\Rishabh\Motu_Ready_Files\falling\freesound_025_0004\ch_4.wav
Wav File : ..\Rishabh\Motu_R

# END!